# Create QR codes of important government documents

Here is a Python function that takes a JSON string, encodes it in base64, compresses it with zlib, encrypts and signs it with a private key, and finally converts it to a QR code:

In [1]:
import base64
import zlib
import qrcode
from cryptography.fernet import Fernet
from cryptography.hazmat.primitives.asymmetric import padding, rsa

In [6]:
def encode_json(json_string, private_key):
    # Generate a public/private key pair
    public_key = private_key.public_key()
    
    # Encode the JSON string in base64
    encoded_json = base64.b64encode(json_string.encode())
    
    # Compress the encoded JSON string with zlib
    compressed_json = zlib.compress(encoded_json)
    
    # Encrypt the compressed JSON string with the private key
    encrypted_json = rsa.decrypt(compressed_json, private_key, padding.OAEP)
    
    # Sign the encrypted JSON string with the private key
    signature = rsa.sign(encrypted_json, private_key, padding.PKCS1v15, "SHA-256")
    
    # Concatenate the encrypted JSON string and the signature
    signed_json = encrypted_json + signature
    
    # Convert the signed JSON string to a QR code
    qr = qrcode.QRCode()
    qr.add_data(signed_json)
    qr.make()
    qr_code = qr.make_image()
    
    # Return the QR code and the public key
    return qr_code, public_key

In [7]:
json_string = '{"name": "John", "age": 30}'
private_key = rsa.generate_private_key(
    public_exponent=65537,
    key_size=2048
)
qr_code, public_key = encode_json(json_string, private_key)

AttributeError: module 'cryptography.hazmat.primitives.asymmetric.rsa' has no attribute 'decrypt'

In [5]:
def decode_json(qr_code, public_key):
    # Read the data from the QR code
    data = qrcode.decode(qr_code)
    
    # Split the data into the encrypted JSON string and the signature
    encrypted_json = data[:256]
    signature = data[256:]
    
    # Verify the signature with the public key
    try:
        rsa.verify(encrypted_json, signature, public_key, padding.PKCS1v15, "SHA-256")
        is_valid = True
    except Exception:
        is_valid = False
    
    # Decrypt the encrypted JSON string with the public key
    compressed_json = rsa.decrypt(encrypted_json, public_key, padding.OAEP)
    
    # Decompress the compressed JSON string
    encoded_json = zlib.decompress(compressed_json)
    
    # Decode the encoded JSON string
    decoded_json = base64.b64decode(encoded_json).decode()
    
    # Return the decoded JSON string and the proof of validity
    return decoded_json, is_valid

In [ ]:
decoded_json, is_valid = decode_json(qr_code, public_key)

In [23]:
from cryptography.hazmat.primitives.asymmetric import padding, rsa

from cryptography.hazmat.primitives.asymmetric import rsa
#from cryptography.hazmat.primitives import serialization
#from cryptography.hazmat.primitives.asymmetric import padding
#from cryptography.hazmat.primitives import hashes

def encrypt_string(string, private_key):
    # Encode the string as bytes
    data = string.encode()
    
    # Encrypt the data with the private key
    encrypted_data = rsa.decrypt(data, private_key, padding.OAEP)
    
    # Return the encrypted data
    return encrypted_data

# TEST
string = "Hello, world!"
private_key = rsa.generate_private_key(
    public_exponent=65537,
    key_size=2048
)
public_key = private_key.public_key()

print(private_key)
print(public_key)

data = string.encode()
encrypted_data = public_key.encrypt(data, padding.OAEP)

encrypted_data = encrypt_string(string, private_key)

print(encrypted_data)

TypeError: Padding must be an instance of AsymmetricPadding.

In [ ]:
def decrypt_string(encrypted_data, public_key):
    # Decrypt the encrypted data with the public key
    data = rsa.decrypt(encrypted_data, public_key, padding.OAEP)
    
    # Decode the data and return the decrypted string
    return data.decode()

decrypted_string = decrypt_string(encrypted_data, public_key)
print(decrypted_string)